In [2]:
import json
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import numpy as np

In [3]:
import json
import os
import numpy as np

def convert_coco_to_yolo(coco_annotations_path, images_dir, output_dir):
    with open(coco_annotations_path, 'r') as f:
        data = json.load(f)

    images = {image['id']: image for image in data['images']}
    annotations = data['annotations']
    categories = {category['id']: category for category in data['categories']}

    category_name_to_id = {category['name']: idx for idx, category in enumerate(categories.values())}

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)

    for annotation in annotations:
        image_id = annotation['image_id']
        image_info = images[image_id]
        image_filename = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            continue

        output_image_path = os.path.join(output_dir, 'images', image_filename)
        output_label_path = os.path.join(output_dir, 'labels', os.path.splitext(image_filename)[0] + '.txt')

        os.system(f'cp "{image_path}" "{output_image_path}"')

        with open(output_label_path, 'a') as f:
            category_name = categories[annotation['category_id']]['name']
            category_id = category_name_to_id[category_name]
            
            # Normalize bounding box coordinates
            bbox = annotation['bbox']
            bbox[0] /= image_width  # x
            bbox[1] /= image_height # y
            bbox[2] /= image_width  # width
            bbox[3] /= image_height # height
            bbox_str = ' '.join(map(str, bbox))

            # Normalize segmentation points
            segmentation = annotation['segmentation'][0]
            segmentation = np.array(segmentation).reshape((-1, 2))
            segmentation[:, 0] /= image_width   # x points
            segmentation[:, 1] /= image_height  # y points
            segmentation_str = ' '.join(map(str, segmentation.flatten()))

            f.write(f"{category_id} {bbox_str} {segmentation_str}\n")


In [4]:
# Paths to your data
coco_annotations_path = 'SAF22_COCO_CorentinDoche.json'
images_dir = 'images'  # Change this to your image directory
output_dir = 'output'  # Directory where YOLO formatted data will be saved

convert_coco_to_yolo(coco_annotations_path, images_dir, output_dir)

In [5]:
from ultralytics import YOLO

In [6]:
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

In [10]:
# Train the model
#results = model.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=50, imgsz=640)
results = model.train(data="C:/Users/ds678/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=50, imgsz=640)


Ultralytics YOLOv8.2.58  Python-3.11.4 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i9-12900)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/ds678/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

RuntimeError: Dataset 'C://Users/ds678/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml' error  
Dataset 'C://Users/ds678/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml' images not found , missing path 'C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\images\val'
Note dataset download directory is 'C:\Users\ds678\OneDrive - University of Sussex\datasets'. You can update this in 'C:\Users\ds678\AppData\Roaming\Ultralytics\settings.yaml'

In [ ]:
print(results.class_result(0))
print(results.class_result(1))
print(results.class_result(2))
print(results.class_result(3))